In [21]:
#TODO: create customed anno, anno_dict from raw csv
#input is csv annotation file
#output is anno, anno_dict

import pandas as pd
import cv2
import numpy as np
import glob
import os
import pickle

import namespace
import log_parser
import youtubebb_converter
import imagenet_converter
import dataset_generator
import metric_map
import predictor
import utils


reload(utils)
reload(log_parser)
reload(namespace)
reload(youtubebb_converter)
reload(predictor)
reload(dataset_generator)

from matplotlib import pyplot as plt
%matplotlib inline



def plot_bbox(rgb_img, pred_item, color_idx=0):
    _, cid, score, xmin, ymin, xmax, ymax = pred_item
    cname = namespace.CLASS_INDEX[cid]
    
    scale = np.array(rgb_img.shape[1::-1]+rgb_img.shape[1::-1])#[1::-1])
    label_name = cname
    display_txt = '%s: %.2f'%(label_name, score)
    detection = np.array([xmin, ymin, xmax, ymax])
    pt = detection * scale#(detection*scale).cpu().numpy()
    coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
    
    colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
    color = colors[0]

    plt.imshow(rgb_img)
    
    if len(pred_item) > 0:
        currentAxis = plt.gca()
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
    plt.axis('off')
    plt.figure()
    
def get_groundtruth_item(img_filepath, vid_filepath, pred):
    #TODO: 
    img = cv2.imread(img_filepath, cv2.IMREAD_COLOR)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    dtoken, ftoken = utils.get_dirtoken_from_vidpath(vid_filepath), utils.get_filetoken_from_imgpath(vid_filepath, img_filepath)
    gt_list = yanno_dict[dtoken][ftoken]
    
    pred_list = pred.detect(img)
    if pred_list == []:
        #print ('DETECTED NOTHING for {}, SKIPPED'.format(img_filepath))
        return 0.0#gt_list[0], pred_list, 0.0
    #plot_bbox(rgb_img, pred_list[0])
    #plot_bbox(rgb_img, gt_list[0], color_idx=10)

    score = mAP.score(pred_list, gt_list)
    return score#gt_list[0], pred_list, score

def get_all_frameversions(frame_filepath, vid_filepath, pred):
    #TODO: for a given frame in max bitrate & max resolution (x1080_b1024), 
    #retrive corresponding frames in other bitrates & resolutions
    #INPUT: a filepath to frame in highest res & bit
    #OUTPUT: grouthtruth for everyone
    
    frame_filemask = frame_filepath.replace(namespace.FRAMEVERSION_BEST, '*')
    result = []
    for img_filepath in glob.glob(frame_filemask):
        result.append(get_groundtruth_item(img_filepath, vid_filepath, pred))
    return result

def generate_viditem_groundtruth(vidpath, pred):
    #TODO: generate ground truth from given video path
    #INPUT: path to video, predictor
    #OUTPUT: dict structure store groundtruth all frames for that video
    result = {}
    framemask = os.path.splitext(vidpath)[0] + '/' + namespace.FRAMEVERSION_BEST + '/??????.jpg'
    for framepath in glob.glob(framemask): #go over all frame item in the best verion
        #for each best frame item, collect score over all of its versions
        #print item
        try:
            gt_list = get_all_frameversions(framepath, vidpath, pred)
        except:
            print 'ERROR: ', framepath, vidpath
            raise
        result[framepath] = gt_list
    return vidpath, result

In [13]:
#in order for log_parser to work, previous steps are assumed to be taken.
#see https://github.com/phananh1010/content-aware/tree/master/scripts for further detail

LogParser = log_parser.LogParser()
vidinfo_dict= LogParser.load_metainfo_dict()

YConverter = youtubebb_converter.YoutubeBBConverter(vidinfo_dict)
mAP = metric_map.mAP()
Pred  = predictor.Predictor()

#only run once to create processed Youtube annotation dict, write to FILEPATH_YOUTUBE_YANNODICT
#YConverter.parse_annotation(namespace.FILEPATH_YOUTUBE_RAWANNOCSV, namespace.FILEPATH_YOUTUBE_YANNODICT)
yanno, yanno_dict = YConverter.load_annotation(namespace.FILEPATH_YOUTUBE_YANNODICT)
DatGen = dataset_generator.DatasetGenerator(YConverter, mAP, Pred, yanno_dict)

#for debuging purpose only
df = pd.DataFrame.from_csv('./data/YOUTUBE_data/yt_bb_detection_train.csv', header=None)
df1 = pd.read_pickle('./data/YOUTUBE_data/yt_bb_detection_train_filtered.pkl.gz', compression='gzip')
                            #./data/YOUTUBE_data/yt_bb_detection_train_filtered.csv


Loading weights into state dict...
Finished!


/home/u9167/.conda/envs/env_torch/lib/python2.7/site-packages/ipykernel_launcher.py:17: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [18]:
#TODO
#from ID=15, list all videos (use bash to generate)
#from videos list, call python code to generate groundtruth

#the python code: receive a filepath to the videos, iterate over images
#ID = 15
#PYTHON CODE
#INPUT: filepath to video
#OUTPUT: ground truth array for every single frame


In [22]:
namespace.DIRPATH_YOUTUBE_VIDEOS

'/home/u9167/content_aware/data/YOUTUBE_data/videos'

In [36]:
ID = 15

_GROUNDTRUTH_DICT_TEMPLATE_ = namespace.DIRPATH_YOUTUBE_DATA + '/' + '_groundtruth_dict_{}_'
GROUNDTRUTH_DICT_FILEPATH = _GROUNDTRUTH_DICT_TEMPLATE_.format(ID)
vidmask = namespace.DIRPATH_YOUTUBE_VIDEOS + '/{}/'.format(ID) + '*.mp4'

gt_dict = {}
try:
    gt_dict = pickle.load(open(GROUNDTRUTH_DICT_FILEPATH))
except:
    print 'ERROR, no file found, will use default empty dic'
    

for vid_filepath in glob.glob(vidmask):
    if vid_filepath in gt_dict: 
        print ("SKIPPED {}".format(vid_filepath))
        continue
    print ('processing {}'.format(vid_filepath))
    k, v = generate_viditem_groundtruth(vid_filepath, Pred)
    gt_dict[k] = v

SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/3zcr2YpUk1M+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/43TG-8oaSXg+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/4Ht9Cn1u9sw+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/4UtuV5h1IWY+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/4z93xWvsjNk+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/6kgOuDfwx6Q+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/7m_Fvo-uWK8+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/AdEH-oHs1Qo+15+0.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/AdEH-oHs1Qo+15+1.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/AdEH-oHs1Qo+15+2.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/AdEH-oHs1Qo+15+3.mp4
SKIPPED /home/u9167/content_aware/data/YOUTUBE_data/videos/15/Es46YsO9J5M+15+0.mp4
SKIP

In [38]:
pickle.dump(gt_dict, open(GROUNDTRUTH_DICT_FILEPATH, 'w'))

In [ ]:
#####################END OF EXAMPLE USAGE#######

In [ ]:
#####################DEBUGGING PART###############

In [4]:
vidpath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0.mp4'

#given video, output frames

framemask = os.path.splitext(vidpath)[0] + '/' + namespace.FRAMEVERSION_BEST + '/*.jpg'
for item in glob.glob(framemask):
    print item
#framemask_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0.mp4'



/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000001.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000002.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000003.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000004.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000005.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000006.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000007.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000008.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000009.jpg
/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000010.jpg


In [6]:
frame_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0/frames_x1080_b1024k/000003.jpg'
vid_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/1/7sq8O2KpCf0+1+0.mp4'

vid_filepath = './data/YOUTUBE_data/videos/15/7m_Fvo-uWK8+15+0.mp4'
frame_filepath = './data/YOUTUBE_data/videos/15/7m_Fvo-uWK8+15+0/frames_x1080_b1024k/000003.jpg'

frame_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/10/8q0FNaWFBjw+10+0/frames_x1080_b1024k/000005.jpg'
vid_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/10/8q0FNaWFBjw+10+0/'

frame_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/19/ShDBzVI0_Io+19+0/frames_x1080_b1024k/000005.jpg'
vid_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/19/ShDBzVI0_Io+19+0.mp4'


#given framepath & vidpath, generate groundtruth
get_all_frameversions(frame_filepath, vid_filepath, Pred)

[1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.1,
 0.0,
 1.0,
 0.1,
 0.1,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1,
 0.1,
 0.1,
 0.0,
 0.0,
 0.0,
 1.0,
 0.1,
 0.1,
 0.1,
 0.0,
 0.0,
 0.1,
 0.0,
 0.0,
 0.1,
 0.1,
 0.0]

In [24]:
img_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/15/4Ht9Cn1u9sw+15+0/frames_x1080_b1024k/10000001.jpg'
vid_filepath = '/home/u9167/content_aware/data/YOUTUBE_data/videos/15/4Ht9Cn1u9sw+15+0.mp4'

utils.get_filetoken_from_imgpath(vid_filepath, img_filepath)

'4Ht9Cn1u9sw+9+0+10000001'